In [ ]:
pip install pandas

In [ ]:
pip install nltk


In [1]:
import pandas as pd
df = pd.read_csv('Historical Lead Records.csv', encoding='latin1')

df.head()

,Title,Job Role,Job Function,Job Level
0,Manager-Cybersecurity,Information Security,IT,Manager
1,"Manager, Information Security",Information Security,IT,Manager
2,User Experience Analyst,Development,Engineering,Contributor
3,Network Specialist,Networking,IT,Contributor
4,Director of Privacy and Compliance,Information Security,IT,Director


In [3]:
#make title column lowercase
df['Title'] = df['Title'].str.lower()

df.shape[0]

865671

In [4]:
na_counts = df.isna().sum()
print(na_counts)

Title           12682
Job Role         4502
Job Function     5083
Job Level       24546
dtype: int64


In [5]:
#1616 rows are completely empty
before_dropping = df.shape[0]
empty_rows= df.isna().all(axis=1).sum()
df = df.dropna(how='all')
after_dropping = df.shape[0]


print (before_dropping)
print (empty_rows)
print (after_dropping)

865671
1616
864055


In [ ]:
question_mark = df['Title'].str.contains('\?').sum()
question_mark

In [ ]:
# #replacing all question marks with empty strings
# df['Title'] = df['Title'].str.replace('?', '')
# question_mark1 = df['Title'].str.contains('\?').sum()
# question_mark1

In [6]:
df = df.dropna(subset=['Title'])
df.shape[0]

852989

In [8]:
import pandas as pd
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from joblib import Parallel, delayed
import numpy as np

def is_english(text):
    try:
        return detect(str(text)) == 'en'
    except LangDetectException:
        return False

def filter_english_titles(df):
    return df[df['Title'].apply(is_english)]

num_jobs = 4
chunks = np.array_split(df, num_jobs)
df_filtered_chunks = Parallel(n_jobs=num_jobs)(
    delayed(filter_english_titles)(chunk) for chunk in chunks
)
df_filtered = pd.concat(df_filtered_chunks)



In [9]:
df_filtered.shape[0]

550328

In [10]:
df_filtered.head()

,Title,Job Role,Job Function,Job Level
0,manager-cybersecurity,Information Security,IT,Manager
1,"manager, information security",Information Security,IT,Manager
2,user experience analyst,Development,Engineering,Contributor
3,network specialist,Networking,IT,Contributor
4,director of privacy and compliance,Information Security,IT,Director


In [19]:
def replace_role(row):
    if pd.isnull(row['Job Function']): 
        return 'N/A'
    elif 'IT' not in row['Job Function']:
        return 'N/A'
    else:
        return row['Job Role']

df_filtered['Job Role'] = df_filtered.apply(replace_role, axis=1)

In [22]:
df_filtered['Job Function'] = df_filtered['Job Function'].fillna('')

non_it = df_filtered[~df_filtered['Job Function'].str.contains('IT')]
non_it.head()

,Title,Job Role,Job Function,Job Level
2,user experience analyst,N/A,Engineering,Contributor
14,"branch manager, purchasing",N/A,Purchasing,Manager
18,legislative affairs director,N/A,Legal,Director
25,associate director of software development,N/A,Engineering,Director
28,turning vision into reality,N/A,Engineering,Unknown


In [ ]:
# delete rows where a title column is empty?
#rows with question marks?

In [ ]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [ ]:
job_levels = df['Job Level'].value_counts()
job_functions = df['Job Function'].value_counts()

print(job_levels.head())
print(job_functions.head())